In [1]:
!pip install impyute
!pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
import impyute as impy
from fancyimpute import IterativeSVD, SoftImpute, NuclearNormMinimization
import pandas as pd
import time

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 87.4 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 91.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.2 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29881 sha256=edb62b55996d2a6864a9f873e8c58a5e36efda9d8fdaab4968939ac14d4521db
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11330 sha256=aae6e6122ec77b1ae8d2d52b275ed4f8e6905caa89f803c4eb7dedf0395c0bf8
  Stor

In [2]:
import numpy as np

def mle(Xtrain, n, p, G):
    '''
    Xtrain: List of input. The ith element of the list contains samples from the ith class.
    Adjusted for the health-related dataset.
    '''
    if p[0] == 1:
        mus = [np.mean(Xtrain[g][:, 0]) for g in np.arange(G)]
        S = [(n[g, 0] - 1) * np.var(Xtrain[g][:, 0]) for g in np.arange(G)]
    else:
        mus = [np.mean(Xtrain[g][:, 0:p[0]], axis=0) for g in np.arange(G)]
        S = [(n[g, 0] - 1) * np.cov(Xtrain[g][:, 0:p[0]], rowvar=False) for g in np.arange(G)]

    mus = np.asarray(mus).T
    S = sum(S) / (sum(n[:, 0]))
    S = S.reshape((p[0], -1))

    for i in np.arange(1, len(p)):
        W = [(n[g, i] - 1) * np.cov(Xtrain[g][0:n[g, i], 0:p[i]], rowvar=False) for g in np.arange(G)]
        W = sum(W)

        P = np.matmul(W[(p[i - 1]):p[i], 0:p[i - 1]], np.linalg.inv(W[0:p[i - 1], 0:p[i - 1]]))
        Q = (W[p[i - 1]:p[i], p[i - 1]:p[i]] - np.matmul(P, W[0:p[i - 1], p[i - 1]:p[i]])) / sum(n[:, i])

        xmeans = [np.mean(Xtrain[g][0:n[g, i], 0:p[i]], axis=0) for g in np.arange(G)]
        xmeans = np.asarray(xmeans).T

        mus = np.vstack((mus, xmeans[p[i - 1]:p[i], :] - np.matmul(P, xmeans[0:p[i - 1]] - mus)))
        S21 = np.matmul(P, S)
        S = np.vstack((np.hstack((S, S21.T)), np.hstack((S21, Q + np.matmul(P, S21.T)))))

    return [mus, S]

In [3]:
# Function to calculate misclassification rate for LDA with missing data
def lda_miss(mus, S, Xtest, ytrain, ytest, G):
    f = lambda g: np.log(np.mean(ytrain == g)) - np.matmul(
                  np.matmul(mus[:,g].T, np.linalg.inv(S)), mus[:,g]/2)
    last2 = [f(g) for g in np.arange(G)]

    h = lambda g, i: last2[g] + np.matmul(mus[:, g].T, np.matmul(
                    np.linalg.inv(S), Xtest[i, :].T))

    pred_label = [np.argmax([h(g, i) for g in np.arange(G)])
                  for i in np.arange(len(Xtest))]

    pred_label = np.asarray(pred_label)
    return np.mean(pred_label.flatten() != ytest)

# Function to create data list with missing values
def make_nan_list(X, y, G, n, p):
    # Labels should range from 0 to G-1
    data = []
    for g in np.arange(G):
        data.append(X[y == g, :])
        for k in np.arange(len(p) - 1):
            data[g][n[g, k + 1]:n[g, k], p[k]:] = np.nan
    return data

In [4]:
def missing_rate(Xtrain, ytrain, n, p, G):
    Xtr_nan_list = make_nan_list(Xtrain, ytrain, G, n, p)
    # Generate missing data and a new ytrain since the order might change
    Xtr_nan, ytr = Xtr_nan_list[0], np.repeat(0, len(Xtr_nan_list[0]))

    for g in np.arange(1, G):
        Xtr_nan = np.vstack((Xtr_nan, Xtr_nan_list[g]))
        ytr = np.hstack((ytr, np.repeat(g, len(Xtr_nan_list[g]))))

    # Calculate the percentage of missing values
    per_missing = np.mean(np.isnan(Xtr_nan))
    return per_missing

In [5]:
def compute_err_MLE(Xtrain, ytrain, Xtest, ytest, n, p, G):    
    Xtr_nan_list = make_nan_list(Xtrain,ytrain,G, n, p)
    # make NA data
    # since making function changes the order of observation
    # we need to generate new ytr from Xtr_nan    
    Xtr_nan, ytr = Xtr_nan_list[0], np.repeat(0, len(Xtr_nan_list[0]))
    for g in np.arange(1,G):
        Xtr_nan = np.vstack((Xtr_nan, Xtr_nan_list[g]))
        ytr = np.hstack((ytr, np.repeat(g, len(Xtr_nan_list[g]))))

    # percentage of missing values
    per_missing = np.mean(np.isnan(Xtr_nan))

    scaler = MinMaxScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtest = scaler.transform(Xtest)
    Xtr_nan_list2 = []
    for g in range(G):
      Xtr_nan_list2.append(scaler.transform(Xtr_nan_list[g]))

    # MLEs approach
    start = time.time()
    mus, S = mle(Xtr_nan_list2, n, p, G)
    mle_err = lda_miss(mus, S, Xtest, ytrain, ytest, G)
    mle_time = time.time()-start
  
    return mle_err, mle_time

In [8]:
import pandas as pd

# Load your dataset instead of MNIST
data = pd.read_csv('/kaggle/input/large-diab/large_diab.csv')

# Split into train and test sets if not already done
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Extract features and target
Xtrain = train_data.drop('HeartDiseaseorAttack', axis=1)  # Assuming 'diabetes_status' is the target
ytrain = train_data['HeartDiseaseorAttack']
Xtest = test_data.drop('HeartDiseaseorAttack', axis=1)
ytest = test_data['HeartDiseaseorAttack']

In [9]:
data.info

<bound method DataFrame.info of         HeartDiseaseorAttack  HighBP  HighChol  CholCheck   BMI  Smoker  \
0                        0.0     1.0       1.0        1.0  40.0     1.0   
1                        0.0     0.0       0.0        0.0  25.0     1.0   
2                        0.0     1.0       1.0        1.0  28.0     0.0   
3                        0.0     1.0       0.0        1.0  27.0     0.0   
4                        0.0     1.0       1.0        1.0  24.0     0.0   
...                      ...     ...       ...        ...   ...     ...   
253675                   0.0     1.0       1.0        1.0  45.0     0.0   
253676                   0.0     1.0       1.0        1.0  18.0     0.0   
253677                   0.0     0.0       0.0        1.0  28.0     0.0   
253678                   0.0     1.0       0.0        1.0  23.0     0.0   
253679                   1.0     1.0       1.0        1.0  25.0     0.0   

        Stroke  Diabetes  PhysActivity  Fruits  ...  AnyHealthcare 

In [10]:
# Convert the dataset to NumPy arrays
Xtrain_np, ytrain_np = [], []
for index, row in train_data.iterrows():
    Xtrain_np.append(row.drop('HeartDiseaseorAttack').values)  # Use all features except the target
    ytrain_np.append(row['HeartDiseaseorAttack'])

Xtrain, ytrain = np.asarray(Xtrain_np), np.asarray(ytrain_np)

# Set random seed and shuffle the data
np.random.seed(1)
idx = np.arange(len(ytrain))
np.random.shuffle(idx)
Xtrain, ytrain = Xtrain[idx, :], ytrain[idx]

Xtrain.shape, ytrain.shape

((202944, 21), (202944,))

In [11]:
# Convert the test set to NumPy arrays
Xtest_np, ytest_np = [], []
for index, row in test_data.iterrows():
    Xtest_np.append(row.drop('HeartDiseaseorAttack').values)  # Use all features except the target
    ytest_np.append(row['HeartDiseaseorAttack'])

Xtest, ytest = np.asarray(Xtest_np), np.asarray(ytest_np)

Xtest = Xtest.astype(float)

In [12]:
# Check if a column is mostly zeros (all or almost all zeroes)
id = [np.sum(Xtrain[:, i] != 0) > 10 for i in range(Xtrain.shape[1])]

# Number of columns that are mostly zero
print(Xtrain.shape[1] - np.sum(id))

# Number of columns with more than 10 non-zero values
print(np.sum(id))

0
21


In [13]:
Xtrain, Xtest = Xtrain[:,id], Xtest[:,id]

In [14]:
Xtrain.shape, Xtest.shape

((202944, 21), (50736, 21))

In [15]:
# Number of samples per class in training data (assuming classes 0 and 1 for diabetes classification)
ng = np.asarray([sum(ytrain == i) for i in np.unique(ytrain)])
ng

array([183819,  19125])

In [16]:
# Update 'n' using the actual class distribution for 20% missingness
n = np.hstack((ng.reshape((-1, 1)), np.tile([200000, 180000, 160000, 140000], len(np.unique(ytrain))).reshape((len(np.unique(ytrain)), -1))))

# Update 'p' using the actual number of features in your dataset
p = np.array([10, 12, 15, 18, Xtrain.shape[1]])

# Calculate the missing rate with updated parameters
missing_rate_value = missing_rate(Xtrain, ytrain, n, p, len(np.unique(ytrain)))
missing_rate_value

0.05030029395864306

In [17]:
compute_err_MLE(Xtrain, ytrain, Xtest, ytest, n, p, len(np.unique(ytrain)))

(0.09803689687795648, 11.92579174041748)

In [18]:
# Update 'n' using the actual class distribution for 30% missingness
n = np.hstack((ng.reshape((-1, 1)), np.tile([140000, 120000, 110000, 90000], len(np.unique(ytrain))).reshape((len(np.unique(ytrain)), -1))))

# Update 'p' using the actual number of features in your dataset
p = np.array([10, 12, 15, 18, Xtrain.shape[1]])

# Calculate the missing rate with updated parameters
missing_rate_value_30 = missing_rate(Xtrain, ytrain, n, p, len(np.unique(ytrain)))
missing_rate_value_30

0.18349162236638586

In [19]:
compute_err_MLE(Xtrain, ytrain, Xtest, ytest, n, p, len(np.unique(ytrain)))

(0.09809602649006623, 12.982503890991211)

In [20]:
# Update 'n' using the actual class distribution for 40% missingness
n = np.hstack((ng.reshape((-1, 1)), np.tile([120000, 100000, 80000, 60000], len(np.unique(ytrain))).reshape((len(np.unique(ytrain)), -1))))

# Update 'p' using the actual number of features in your dataset
p = np.array([10, 12, 15, 18, Xtrain.shape[1]])

# Calculate the missing rate with updated parameters
missing_rate_value_40 = missing_rate(Xtrain, ytrain, n, p, len(np.unique(ytrain)))
missing_rate_value_40

0.24919119137721313

In [21]:
compute_err_MLE(Xtrain, ytrain, Xtest, ytest, n, p, len(np.unique(ytrain)))

(0.09797776726584674, 11.686202764511108)

In [22]:
# Update 'n' using the actual class distribution for 50% missingness
n = np.hstack((ng.reshape((-1, 1)), np.tile([100000, 90000, 80000, 70000], len(np.unique(ytrain))).reshape((len(np.unique(ytrain)), -1))))

# Update 'p' using the actual number of features in your dataset
p = np.array([10, 12, 15, 18, Xtrain.shape[1]])

# Calculate the missing rate with updated parameters
missing_rate_value_50 = missing_rate(Xtrain, ytrain, n, p, len(np.unique(ytrain)))
missing_rate_value_50

0.2585768440930456

In [23]:
compute_err_MLE(Xtrain, ytrain, Xtest, ytest, n, p, len(np.unique(ytrain)))

(0.09801718700725323, 13.409820318222046)